# Introduction


Use this notebook as a `changelog` notebook for setting `REFERENCE` feature-set in measuring feature **drifts**. 

The first version is `20240925`. In this version, we use all `training` data feature as **reference**. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pathlib
from tqdm import tqdm
from feature_class import *

data_version = "20240924"
reference_version = "20240925"

path_to_feature_dir = f"/Users/hieunguyen/data/WGS_features/{data_version}/feature"
path_to_metadata = f"/Users/hieunguyen/data/WGS_features/{data_version}/metadata/metadata_nonBS_200924.xlsx"

path_to_save_output = "feature_drift_reference/OT"
os.system(f"mkdir -p {path_to_save_output}")

obj = WGS_GW_features(path_to_feature_dir=path_to_feature_dir,
                      path_to_metadata=path_to_metadata)
